In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("Bank_Churn.csv")

In [3]:
df.columns

Index(['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age',
       'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

In [4]:
df.isna().sum()

CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [5]:
df.duplicated().sum()

0

In [6]:
df.dtypes

CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [30]:
df=pd.read_csv("Bank_Churn.csv")
db_path =r"C:\Users\USER\Desktop\My Work\Database-SQLite\Bank.db"
conn =sqlite3.connect(db_path)
cursor =conn.cursor()
table = """
CREATE TABLE IF NOT EXISTS Clients_Profile (
    CustomerId INTEGER PRIMARY KEY,
    Surname TEXT,
    CreditScore INTEGER,
    Geography TEXT,
    Gender TEXT,
    Age INTEGER,
    Tenure INTEGER,
    Balance INTEGER,
    NumOfProducts INTEGER,
    HasCrCard INTEGER,
    IsActiveMember REAL,
    EstimatedSalary INTEGER,
    Exited INTEGER
);
"""
cursor.execute(table)
conn.commit()
print("Table 'Clients_Profile' created successfully")
print("===" * 20)
df.to_sql("Clients_Profile",conn,if_exists="replace",index=False)
print("Data uploaded to 'Clients_Profile' table successfully!")

Table 'Clients_Profile' created successfully
Data uploaded to 'Clients_Profile' table successfully!


#### a) Lets view the first 5 rows of the table

In [8]:
query ="SELECT* FROM Clients_Profile LIMIT 5"
dff =pd.read_sql(query,conn)
display(dff.head(5))

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### b) How many clients does the Bank serve?

In [9]:
cursor.execute("SELECT COUNT(DISTINCT CustomerId) FROM Clients_Profile")
distinct_count =cursor.fetchone()[0]
print(f"The bank serves {distinct_count} customers")

The bank serves 10000 customers


#### c) Count of Customers by Geography?

In [11]:
query ="""
SELECT Geography, COUNT(*) as Customer_Count
FROM Clients_Profile
GROUP BY Geography
ORDER BY Customer_Count DESC
"""
dff =pd.read_sql(query,conn)
display(dff)

,Geography,Customer_Count
0,France,5014
1,Germany,2509
2,Spain,2477


#### d) Count of Customers by Gender

In [14]:
query ="""
SELECT Gender, COUNT(*) as Customer_Count
FROM Clients_Profile
GROUP BY Gender
ORDER BY Customer_Count DESC
"""
dff =pd.read_sql(query,conn)
display(dff)

,Gender,Customer_Count
0,Male,5457
1,Female,4543


#### e) How many customers have Exited/Churned and whats the Churn Rate?

In [10]:
cursor.execute("SELECT * FROM Clients_Profile WHERE Exited = 1")
exited =len(cursor.fetchall())
print(f"Out of {distinct_count} customers, {exited} have exited the bank")
print()
print(f"Churn Rate: {((exited/10000) * 100):.2f}%")

Out of 10000 customers, 2037 have exited the bank

Churn Rate: 20.37%


#### f) Churn rate by Gender

In [16]:
query ="""
SELECT Gender,COUNT(*) as Churned_Count
FROM Clients_Profile
WHERE Exited = 1
GROUP BY Gender
ORDER BY Churned_Count DESC
"""
dff =pd.read_sql(query,conn)
display(dff.head())

,Gender,Churned_Count
0,Female,1139
1,Male,898


In [17]:
query ="""
SELECT Geography,COUNT(*) as Churned_Count
FROM Clients_Profile
WHERE Exited = 1
GROUP BY Geography
ORDER BY Churned_Count DESC
"""
dff =pd.read_sql(query,conn)
display(dff.head())

,Geography,Churned_Count
0,Germany,814
1,France,810
2,Spain,413


#### Age Distribution

In [22]:
cursor.execute("SELECT MAX(Age) FROM Clients_Profile")
max_age =cursor.fetchone()[0]
cursor.execute("SELECT MIN(Age) FROM Clients_Profile")
min_age =cursor.fetchone()[0]
print(f"\nThe oldest customer is {max_age} years old")
print(f"\nThe youngest customer is {min_age} years old")


The oldest customer is 92 years old

The youngest customer is 18 years old


In [26]:
query ="""
SELECT 
    CASE
        WHEN Age BETWEEN 18 AND 30 THEN '18-30'
        WHEN Age BETWEEN 31 AND 50 THEN '31-50'
        WHEN Age BETWEEN 51 AND 60 THEN '51-60'
        ELSE '60+'
    END AS Age_Group,COUNT(*) AS Churn_Count
FROM Clients_Profile
WHERE Exited = 1
GROUP BY Age_Group
ORDER BY Churn_Count DESC
"""
dff =pd.read_sql(query,conn)
display(dff.head())

,Age_Group,Churn_Count
0,31-50,1326
1,51-60,448
2,18-30,148
3,60+,115


#### Salary Distribution

In [31]:
cursor.execute("SELECT MAX(EstimatedSalary) FROM Clients_Profile")
max_salary =cursor.fetchone()[0]
cursor.execute("SELECT MIN(EstimatedSalary) FROM Clients_Profile")
min_salary =cursor.fetchone()[0]
print(f"\nThe highest salary of a customer is ${max_salary}")
print(f"\nThe lowest salary of a customer is ${min_salary}")


The highest salary of a customer is $199992.48

The lowest salary of a customer is $11.58


In [33]:
query ="""
SELECT
    CASE
        WHEN EstimatedSalary BETWEEN 10 AND 19000 THEN '<20K'
        WHEN EstimatedSalary BETWEEN 20000 AND 49000 THEN '20K-50K'
        WHEN EstimatedSalary BETWEEN 50000 AND 99000 THEN '50K-100K'
        WHEN EstimatedSalary BETWEEN 100000 AND 149000 THEN '100K-150K'
        ELSE '150K-200K'
    END AS Salary_Group,COUNT(*) AS Churn_Count
FROM Clients_Profile
WHERE Exited = 1
GROUP BY Salary_Group
ORDER BY Churn_Count
"""
dff =pd.read_sql(query,conn)
display(dff.head())

,Salary_Group,Churn_Count
0,<20K,190
1,20K-50K,279
2,50K-100K,498
3,100K-150K,503
4,150K-200K,567


#### Credit Score Distribution

In [34]:
cursor.execute("SELECT MAX(CreditScore) FROM Clients_Profile")
max_score =cursor.fetchone()[0]
cursor.execute("SELECT MIN(CreditScore) FROM Clients_Profile")
min_score =cursor.fetchone()[0]
print(f"\nThe highest credit score is {max_score}")
print(f"\nThe lowest credit score is {min_score}")


The highest credit score is 850

The lowest credit score is 350


In [38]:
query ="""
SELECT
    CASE
        WHEN CreditScore BETWEEN 350 AND 499 THEN 'Low'
        WHEN CreditScore BETWEEN 500 AND 699 THEN 'Average'
        ELSE 'High'
    END AS CreditScore_Group,COUNT(*) AS Churn_Count
FROM Clients_Profile
WHERE Exited = 1
GROUP BY CreditScore_Group
ORDER BY Churn_Count DESC
"""
dff =pd.read_sql(query,conn)
display(dff.head())

,CreditScore_Group,Churn_Count
0,Average,1263
1,High,624
2,Low,150


In [40]:
cursor.execute("SELECT MAX(Tenure) FROM Clients_Profile")
max_tenure =cursor.fetchone()[0]
cursor.execute("SELECT MIN(Tenure) FROM Clients_Profile")
min_tenure =cursor.fetchone()[0]
print(f"\nThe longest years a customer has stayed with the bank is {max_tenure}")
print(f"\nThe shortest years a customer has stayed with the bank is {min_tenure}")


The longest years a customer has stayed with the bank is 10

The shortest years a customer has stayed with the bank is 0


In [43]:
query ="""
SELECT
    CASE
        WHEN Tenure BETWEEN 0 AND 3 THEN '0-3'
        WHEN Tenure BETWEEN 4 AND 7 THEN '4-7'
        ELSE '8-10'
    END AS Tenure_Group,COUNT(*) AS Churn_Count
FROM Clients_Profile
WHERE Exited = 1
GROUP BY Tenure_Group
ORDER BY Churn_Count DESC
"""
dff =pd.read_sql(query,conn)
display(dff)

,Tenure_Group,Churn_Count
0,4-7,785
1,0-3,741
2,8-10,511


#### Number of Products

In [44]:
cursor.execute("SELECT MAX(NumOfProducts) FROM Clients_Profile")
nums =cursor.fetchone()[0]
print(f"The bank offers {nums} types of products")

The bank offers 4 types of products


In [48]:
query ="""
SELECT NumOfProducts,COUNT(*) as Churn_Count
FROM Clients_Profile
WHERE Exited = 1 
GROUP BY NumOfProducts
ORDER BY Churn_Count DESC
"""
dff =pd.read_sql(query,conn)
display(dff.head())

,NumOfProducts,Churn_Count
0,1,1409
1,2,348
2,3,220
3,4,60


#### Has Credit Card

In [50]:
query ="""
SELECT HasCrCard,COUNT(*) AS Churn_Count
FROM Clients_Profile
WHERE Exited = 1
GROUP BY HasCrCard
ORDER BY Churn_Count DESC
"""
dff =pd.read_sql(query,conn)
display(dff.head())

,HasCrCard,Churn_Count
0,1,1424
1,0,613


### Is Active Members

In [51]:
query ="""
SELECT IsActiveMember,COUNT(*) AS Churn_Count
FROM Clients_Profile
WHERE Exited = 1
GROUP BY IsActiveMember
ORDER BY Churn_Count DESC
"""
dff =pd.read_sql(query,conn)
display(dff.head())

,IsActiveMember,Churn_Count
0,0,1302
1,1,735


In [53]:
query ="""
SELECT NumOfProducts,
       COUNT(*) as TotalCustomers,
       SUM(Exited) as ChurnedCustomers,
       ROUND(100.0 * SUM(Exited) / COUNT(*), 2) as ChurnRate
FROM Clients_Profile
GROUP BY NumOfProducts
ORDER BY ChurnRate DESC
"""
dff =pd.read_sql(query,conn)
display(dff.head())

,NumOfProducts,TotalCustomers,ChurnedCustomers,ChurnRate
0,4,60,60,100.00
1,3,266,220,82.71
2,1,5084,1409,27.71
3,2,4590,348,7.58


In [13]:
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [18]:
df["Age"].max()

92

In [19]:
df["Age"].min()

18